In [1]:
import os
import time 
import cv2
import numpy as np
from model.yolo_model import YOLO

Using TensorFlow backend.


In [28]:
def process_image(img):
    '''
    Resize, reduce and expand image.
    
    # Argument:
        img: original image
        
    # Returns
        image: ndarray(64,64,3), processed image
        
    '''
    image=cv2.resize(img,(416,416),
                    interpolation=cv2.INTER_CUBIC)
    image=np.array(image,dtype='float32')
    image/=255
    image=np.expand_dims(image,axis=0)
    
    return image

<h4><b>This function takes an original image img as input and performs several processing steps on it.</h4></b>

Resize: The cv2.resize() function resizes the input image to a specified size (416, 416). It uses the cv2.INTER_CUBIC interpolation method, which is a higher quality interpolation method compared to others like cv2.INT ER_LINEAR. L

Data type conversion: The resized image is then converted to a NumPy arr ay with dtype='float32'. This step ensures that the image data type is s uitable for further processing, typically in machine learning or compute r vision tasks.

Normalization: The pixel values of the image are normalized by dividing each pixel value by 255. This step scales the pixel values to the range [0, 1], which is a common practice for neural network input data normali zation.

Dimension expansion: Finally, np.expand_dims() is used to add an extra d imension to the processed image array. This is often necessary when deal ing with batch processing in deep learning frameworks, where the first d imension represents the batch size. In this case, axis e indicates that the new dimension is added as the first dimension.

<b>The function returns the processed image with the specified shape (1.416, 416, 3), where 1 represents the batch size, 416 and 416 represent the image dimensions, and 3 represents the number of color channels (RGB).

In [29]:
def get_classes(file):
    """
    Get classes name 
    
    # Argument:
        file: classes name for database
        
    # Returns:
        class_names: List, classes name
    
    """
    with open(file) as f:
        class_names=f.readlines()
    class_names=[c.strip() for c in class_names]
    
    return class_names

<b>The function is designed to read a file containing class names and return them as a list</b>
Opening the file: The function takes a parameter file, which is the path to the file containing the class names. It then opens this file using a with statement, which ensures that file is properly closed after its suite finishes, even if an exception raised

Reading Class Names: Inside the with block, f.readlines() reads all the lines from the file and returns them as a list of strings. Each string r epresents a class name.

Stripping Newlines: Since readlines() includes the newline character \n at the end of each line, the list of class names may contain trailing wh itespace. The list comprehension [c.strip() for c in class_names] is use d to remove leading and trailing whitespace (including newlines) from ea ch class name.

Returning Class Names: The function then returns the list of class names

<b>Overall, this function is a simple utility for exchanging 

In [30]:
import cv2
import numpy as np

def draw(image, boxes, scores, classes, all_classes):
    '''
    Draw the boxes on the image
    
    # Arguments:
        image: original image
        boxes: ndarray, bounding boxes in (x, y, w, h) format
        scores: ndarray, scores of objects
        classes: ndarray, class indices of detected objects
        all_classes: list of class names
    '''
    for box, score, c1 in zip(boxes, scores, classes):
        x, y, w, h = box
        
        top = max(0, int(np.floor(x + 0.5)))
        left = max(0, int(np.floor(y + 0.5)))
        right = min(image.shape[1], int(np.floor(x + w + 0.5)))
        bottom = min(image.shape[0], int(np.floor(y + h + 0.5)))
        
        # Draw the bounding box
        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        
        # Put the label with score
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[c1], score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)
        
        print('Class: {0}, Score: {1:.2f}'.format(all_classes[c1], score))
        print('Box coordinates (x, y, w, h): {0}'.format(box))
    
    print()


This function is designed to draw bounding boxes around detected objects on an image, along with their class labels and confidence scores.

Iterating Over Detected Objects:

It iterates over each detected object using a for loop and zip(boxe s, scores, classes), where boxes, scores, and classes are arrays contain ing the bounding box coordinates, confidence scores, and class indices o f detected objects, respectively.

Bounding Box Coordinates:

For each detected object, it unpacks the bounding box coordinates (x, y, w, h) from box.

It calculates the top-left (top, left) and bottom-right (right, bott om) coordinates of the bounding box. These coordinates are used to draw

Annotating with Class Label and Score:

It annotates the bounding box with the class label and confidence sc ore using cv2.putText(). The label and score are formatted with the clas s name obtained from all_classes[cl] (where cl is the class index) and t he confidence score. This annotation is placed slightly above the top-le ft corner of the bounding box with a red color (0, 0, 255) 

In [31]:
def detect_image(image,yolo,all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: VOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """

    pimage=process_image(image)
    #The image is first processed into a format that the YOLO model 
    #expects, typically involving resizing, normalizing, and possibly 
    #transforming the image to fit the input shape for the YOLO model.
    
    start = time.time()
    boxes, classes, scores =yolo.predict(pimage, image.shape) 
    end = time.time()
    
    print('time: {0:.2f}s'.format(end - start))
    
# Here, the function calls YOLO's predict method on the processed image
# (pimage). This method returns three things:

# boxes: coordinates of bounding boxes for detected objects.

# classes: the class indices (e.g., '0' for 'person', '1' for 'car', etc

    if boxes is not None:
        draw(image,boxes,scores,classes,all_classes)
        
#IF YOLO detects any objects (i.e., boxes is not None), 
#the draw function is called. This function will typically:
#Draw bounding boxes on the original image.
#Label each box with the class name (using all_classes).
#Optionally include the confidence score
    return image


In [32]:
import os
import cv2

def detect_video(video, yolo, all_classes):
    video_path = os.path.join("C:/AI_New/4-YOLO/YOLOv3/videos","C:\AI_New\4-YOLO\YOLOv3\videos\test")
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)
    
    # Prepare for saving the detected video
    frame_width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
    sz = (frame_width, frame_height)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    
    output_dir = os.path.join("C:/AI_New/4-YOLO/YOLOv3/videos", "res")
    os.makedirs(output_dir, exist_ok=True)
    
    vout = cv2.VideoWriter(os.path.join(output_dir, video), fourcc, 30.0, sz)  # Assuming 30 FPS for output video

    while True:
        res, frame = camera.read()
        
        if not res:
            break
        
        image = detect_image(frame, yolo, all_classes)
        cv2.imshow('detection', image)
        
        vout.write(image)
        
        # Exit on 'Esc' key
        if cv2.waitKey(1) & 0xFF == 27:
            break
    
    # Release resources
    vout.release()
    camera.release()
    cv2.destroyAllWindows()


In [33]:
yolo=YOLO(0.6,0.5)
file="C:/AI_New/4-YOLO/YOLOv3/data/coco_classes.txt"
all_classes=get_classes(file)

### Detecting Image

In [34]:
f="C:/AI_New/4-YOLO/YOLOv3/images/test/person.jpg"
image=cv2.imread("C:/AI_New/4-YOLO/YOLOv3/images/test/person.jpg")
image=detect_image(image,yolo,all_classes)
cv2.imwrite("C:/AI_New/4-YOLO/YOLOv3/images/res/"+f,image)

time: 0.00s



False